# Lab \#8: Promoter Sequence Analysis

### Name:  (add name here)

We start by reading in the *promoters.fasta* file that contains the promoters sequences of 78,850 human genes. The code creates a list named *sequences* that contains the 78,850 sequence records.

In [ ]:
from Bio import SeqIO
import re

with open('promoters.fasta') as handle:
    sequences = SeqIO.parse(handle, 'fasta')
    sequences = list(sequences) # get the first sequence

len(sequences)

The code cell below finds the *id* of the first sequence, which has the format
*hg38_ncbiRefSeqCurated_accession*, where *hg38* denotes that the data comes from the *h*uman *g*enome assembly version 38, and from the curated RefSeq database. The accession number is the unique id for the sequence record in the database. If the accession number begins with an *NM*, the record contains an mRNA sequence; if the accession number begins with an *NR*, it refers to a non-coding RNA (an RNA that is not translated into protein). The code below demonstrates how to get the *id* of the first sequence.

In [ ]:
sequences[0].id

### Question 1 <span style = 'font-size:80%'>[10 points]</span>

Use list comprehension to create a list that contains only the id of each sequence. Then display the first 5 elements of the list. Recall that using slicing, we can write `x[:5]` to get the first 5 elements of the list *x*.

### Question 2 <span style = 'font-size:80%'>[10 points]</span>

Use list comprehension to create a list of sequence records containing only mRNA sequences (sequence records whose headers contain an *NM*), and name this list *sequences*. Display the number of sequence records that remain.

## Identifying promoters containing a TATA box

In our discussion about gene expression (https://gdancik.github.io/CSC-314/data/notes/FromGeneToProtein.pdf), we learned about a conserved promoter element known as a TATA box. Transcription factors bind to the TATA box during the initiation phase of transcription. This TATA box is found in about 40% of eukaryotic promoters. 

The TATA box has a consensus sequence that biologists write as 

$TATAWAWN$,

where $W$ is an A or a T and $N$ is any character.

The function below is designed to return True if the sequence in the sequence record *s* contains a TATA box. However the function looks specifically for *tataaaa*, which is found in many, but not all TATA box elements. Note that the second argument of *re.findall* must be a string, so the code below converts the sequence (stored as a Seq object) to a string using *str(s.seq)*.

### Question 3 <span style = 'font-size:80%'>[10 points]</span>

Modify the function to replace 'tataaa' with a regular expression that corresponds to the consensus sequence above. Note that the sequence data is in lowercase.

In [ ]:
def has_TATA_box (s) :
    '''returns True if the sequence in seq_record 's' contains a TATA box'''
    matches = re.findall('tataaaa', str(s.seq))
    return len(matches) > 0

The code below uses the above function and list comprehension to create a list of sequence *ids* for sequences that contain a TATA box. We then display the percent of sequences containing a TATA box (this is around 3.70% for the sequence *tataaaa*, but around 25% for the consensus sequence above). Note that an *f-string* is used to format the percentage as a decimal with a precision of 2.

In [ ]:
res = [s.id for s in sequences if has_TATA_box(s)]
print(f'Percent of promoters with TATA box: {len(res) / len(sequences) * 100:.2f}%')

## Identifying candidate p53 target genes

The gene *TP53*, which codes for the protein p53, is a tumor suppressor protein sometimes referred to as the "guardian of the genome" because of its involvement in shutting down cell division when something goes wrong (like when DNA gets mutated). Failure to shut down cell division following DNA mutation can result in cancer. In fact, over 50% of solid tumors have mutations in p53.

p53 is a *transcription factor* and regulates gene expression by binding to a gene's promoter. Previous studies (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5291249/) have identified p53 binding sites, which contain 4 copies of the consensus sequence

$RRRCWWGYYY$

where $R$ is a purine (A or G), $W$ is an A or a T, and $Y$ is a pyrmidine (C or T).

### Question 4 <span style = 'font-size:80%'>[10 points]</span>

Modify the function to replace 'gggaagg' with the regular expression corresponding to the consensus sequence above. 

In [ ]:
def has_p53_binding_site(s) :
    ''' returns True if 's' contains 2 or more p53 binding sites'''
    matches = re.findall('gggaagg', str(s.seq))
    return len(matches) > 1

The code below uses the above function and list comprehension to create a list of sequence IDs for sequences that contain at least 2 candidate p53 binding sites, and displays the number of candidate target sequences. 

In [ ]:
targets = [ s.id for s in sequences if has_p53_binding_site(s)]
len(targets)

### Question 5 <span style = 'font-size:80%'>[10 points]</span>

Use list comprehension to create a new *targets* list where each element contains the accession number (and only the accession number) for each target sequence. Note that each target sequence *id* has the format 'hg38_ncbiRefSeqCurated_NM_001145658.3', and the accession number is everything after 'hg38_ncbiRefSeqCurated_', e.g., 'NM_001145658.3'. There are two general ways to do this, by either replacing the beginning of the string with an empty string (''), or using the *split* method. Then iterate through the target list to output the first 10 accession numbers.

### Conclusion

The *targets* list now contains the accession numbers of sequence that are potential targets of p53. Such genes are likely important for cellular growth and may play a role in cancer. The next bioinformatics step (which we cover later) would involve using tools such as the UCSC Table Browser (https://genome.ucsc.edu/) to map the accession numbers to the gene name and obtain information about each gene to prioritize the genes to look at. Biologists would then carry out experiments to see if p53 really does bind to the sequence. 